In [1]:
import h5py
from ipywidgets import IntSlider, interact, Dropdown
import numpy as np
import matplotlib.pylab as plt

from spikelib.fitting import fit_temp_sta
from spikelib.utils import check_groups
%matplotlib inline

# General information about input output

In [2]:
# Data files
intensities = ['nd2-255', 'nd3-255', 'nd4-255', 'nd5-255']
intensity = intensities[3] 
print(intensity)
input_file = '../data/processed_protocols/MR-0092t2_modified_analysis_of_protocols_150um_merge.hdf5'
output_file = '../data/processed_protocols/MR-0092t2_modified_analysis_of_protocols_150um_merge.hdf5'

# Groups for raw data in hdf5 file
temp_group = '/sta/{}/temporal/raw/'.format(intensity)
spatial_group = '/sta/{}/spatial/char/'.format(intensity)
valid = '/sta/{}/valid/'.format(intensity)

# Groups for fit data in hdf5 file
temp_fit_group = '/sta/{}/temporal/fit/'.format(intensity)
temp_pars_group = '/sta/{}/temporal/fit_params/'.format(intensity)

nd5-255


# Load Data

In [3]:
# Load data
raw_dtype = np.dtype([('id', 'U18'), ('temp_raw', 'f8',(30,)), ('snr', 'f8')])
upsampling = 30

with h5py.File(input_file, 'r') as stafile:
    print(stafile[spatial_group].attrs['col_name'])
    temporal_sta = np.zeros(len(stafile[temp_group]), dtype=raw_dtype)
    for kidx, kkey in enumerate(stafile[temp_group]):
        temporal_sta[kidx]['id'] = kkey
        temporal_sta[kidx]['temp_raw'] = stafile[temp_group][kkey][...]
        temporal_sta[kidx]['snr'] = stafile[spatial_group][kkey][-2]
    fps = stafile[temp_group].attrs['fps'][...]
    time_raw = stafile[temp_group].attrs['time']

nsample = time_raw.size
time_fit = np.linspace(time_raw[0], time_raw[-1], nsample*upsampling, endpoint=True)  

angle,semia,semib,x0,y0,snr,frame_max


# Fitting for all units

In [4]:
fit_dtype = np.dtype([('id', 'U18'),
                     ('parameters', 'f8',(5,)),
                     ('temp_fit', 'f8', (nsample*upsampling,)),
                    ])

In [5]:
temp_fit = np.zeros((len(temporal_sta)), dtype=fit_dtype)
no_ajustados = []

for kidx, kcell in enumerate(temporal_sta):
    temp_to_fit = kcell['temp_raw']
    params, tmp_fit = fit_temp_sta(temp_to_fit, time_raw, time_fit)
    
    temp_fit[kidx]['id'] = kcell['id']
    parameters = [params[key] for key in ['amp1', 'amp2', 'tau1', 'tau2', 'n']]
    temp_fit[kidx]['parameters'] = np.asarray(parameters)
    temp_fit[kidx]['temp_fit'] = tmp_fit

/home/cesar/Software/anaconda3/envs/piptest/lib/python2.7/site-packages/spikelib/fitting.py:37: RuntimeWarning: divide by zero encountered in power
  gauss1 = amp1 * (time/tau1)**order * np.exp(-order * (time/tau1-1))
/home/cesar/Software/anaconda3/envs/piptest/lib/python2.7/site-packages/spikelib/fitting.py:38: RuntimeWarning: divide by zero encountered in power
  gauss2 = amp2 * (time/tau2)**order * np.exp(-order * (time/tau2-1))
/home/cesar/Software/anaconda3/envs/piptest/lib/python2.7/site-packages/spikelib/fitting.py:39: RuntimeWarning: invalid value encountered in subtract
  model = gauss1 - gauss2


## Save to hdf5

In [33]:
with h5py.File(output_file, 'r+') as fanalysis:
    check_groups(fanalysis, [temp_fit_group, temp_pars_group])      
    for kunit in temp_fit:
        key = kunit['id']
        if key in fanalysis[temp_fit_group]:
            fanalysis[temp_fit_group+key][...] = kunit['temp_fit']
        else:
            fanalysis[temp_fit_group].create_dataset(key, data=kunit['temp_fit'], 
                                                     dtype=np.float, compression="gzip")
        if key in fanalysis[temp_pars_group]:
            fanalysis[temp_pars_group+key][...] = kunit['parameters']
        else:
            fanalysis[temp_pars_group].create_dataset(key, data=kunit['parameters'],
                                                      dtype=np.float, compression="gzip")            
    
    fanalysis[temp_fit_group].attrs['time'] = time_fit
    fanalysis[temp_pars_group].attrs['col_name'] = u'amp1,amp2,tau1,tau2,n'
    fanalysis[temp_pars_group].attrs['info'] = u'parameters are for a two cascade filter described by Chichilnisky2001'
